# Market access and population

This notebook combines the market access calculations from the global friction surface 2019 with population from WorldPop, and urban/rural calculations based on the JDC calculations to create a complete, flexible workflow for measuring access to features:

### Required input data
1. Destinations
2. ISO3 code (for extent extraction)
3. Travel time thresholds in minutes

### Workflow
1. Calculate travel time to destinations  
   a. Calculate binary travel time layers
2. Calculate urban and rural  
   a. Calculate urban and rural population layers  
3. Combine travel time layers (#1a) with population layers (#2a)
4. Run zonal stats on #3


# TODO
1. Rasters may need to be standardized to each other

In [1]:
import sys, os
import rasterio

import pandas as pd
import geopandas as gpd
import numpy as np
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../../../GOST_Urban")
import src.UrbanRaster as urban

sys.path.append("../../")
import infrasap.market_access as ma
import infrasap.rasterMisc as rMisc
from infrasap.misc import tPrint

In [2]:
#User records
destinations = "/home/public/Data/COUNTRY/PAK/HEALTH/pakistan_health_facilities/Pakistan_Health_Facilities.shp"
iso3 = "PAK"
out_folder = "/home/wb411133/data/Country/PAK/HEALTH/"
thresholds = [30, 60, 120, 180]

if not os.path.exists(out_folder):
    os.makedirs(out_folder)
    
# Read in destinations
inH = gpd.read_file(destinations)


In [13]:
# Read in destinations
inH = gpd.read_file(destinations)

# Filter destinations here, if desired
scenario1 = ['GENERAL HOSPITALS','CHILDREN HOSPITAL','TEHSIL HEADQUARTER HOSPITAL','DISTRICT HEADQUARTER HOSPITAL','AGENCY HEADQUARTER HOSPITAL']
scenario2 = ['GENERAL PHYSICIAN','BASIC HEALTH UNIT','GENERAL HOSPITALS','MATERNITY HOME','RURAL HEALTH CENTER','SPECIALIST','CHILDREN HOSPITAL','MCH CENTRE','SUB-HEALTH CENTER','DIAGNOSTIC CENTRE','TEHSIL HEADQUARTER HOSPITAL','DISTRICT HEADQUARTER HOSPITAL','FAMILY WELFARE CENTER','URBAN HEALTH CENTRE','AGENCY HEADQUARTER HOSPITAL']
out_folder = "/home/wb411133/data/Country/PAK/HEALTH_SCENARIO2/"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

#inH = inH.loc[inH['Category'].isin(scenario2)]
inH['cat1'] = 0
inH['cat2'] = 0

inH.loc[inH['Category'].isin(scenario1), 'cat1'] = 1
inH.loc[inH['Category'].isin(scenario2), 'cat2'] = 1

inH.to_file("/home/wb411133/data/Country/PAK/HEALTH_FACILITIES.shp")

In [4]:
global_friction_surface = "/home/public/Data/GLOBAL/INFRA/FRICTION_2020/2020_motorized_friction_surface.geotiff"
global_population = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/ppp_2020_1km_Aggregated.tif"
inG = rasterio.open(global_friction_surface)
inP = rasterio.open(global_population)

# Read in country bounds
global_bounds = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
admin1 = "/home/public/Data/GLOBAL/ADMIN/Admin1_Polys.shp"
inB = gpd.read_file(global_bounds)
inB = inB.loc[inB['ISO3'] == "PAK"]
inB = inB.to_crs(inG.crs)
inB1 = gpd.read_file(admin1)
inB1 = inB1.loc[inB1['ISO3'] == "PAK"]
inB1 = inB1.to_crs(inG.crs)

# Clip the travel raster to ISO3
out_travel_surface = os.path.join(out_folder, "TRAVEL_SURFACE.tif")
rMisc.clipRaster(inG, inB, out_travel_surface)

# Clip the population raster to ISO3
out_pop_surface = os.path.join(out_folder, "POP_2020_NEW.tif")
rMisc.clipRaster(inP, inB, out_pop_surface)

In [5]:
# create MCP object
inG = rasterio.open(out_travel_surface)
inG_data = inG.read() * 1000
# Correct no data values
inG_data[inG_data < 0] = 99999999
mcp = graph.MCP_Geometric(inG_data[0,:,:])

In [6]:
# Calculate travel time
out_file = os.path.join(out_folder, "HEALTH_TRAVEL_TIME_MINUTES.tif")
facility_cells = ma.get_mcp_dests(inG, inH)
costs, traceback = mcp.find_costs(facility_cells)  
costs[np.isinf(costs)] = 0
costs[np.isnan(costs)] = 0
meta = inG.meta.copy()
meta.update(dtype=costs.dtype)
with rasterio.open(out_file, 'w', **meta) as out:
    out.write_band(1, costs)

# Calculate urban

In [7]:
urban_raster = os.path.join(out_folder, "URBAN.tif")
urban_pop_raster = os.path.join(out_folder, "URBAN_POP.tif")
calc_urban = urban.urbanGriddedPop(out_pop_surface)
urban_extents = calc_urban.calculateUrban(densVal=300, totalPopThresh=5000,
                          raster=urban_raster, raster_pop=urban_pop_raster, 
                          print_message=iso3, verbose=True)

13:48:52	PAK: Read in urban data
13:48:54	PAK: Creating Shape 0
13:49:01	PAK: Creating Shape 1000
13:49:07	PAK: Creating Shape 2000
13:49:14	PAK: Creating Shape 3000


# Combine traveltime and population

In [8]:
cur_thresh = thresholds[0]
tt_raster = rasterio.open(out_file)
pop_raster = rasterio.open(out_pop_surface)
urban_pop = rasterio.open(urban_pop_raster)

tt_d = tt_raster.read()
pop_d = pop_raster.read()
urban_pop_d = urban_pop.read()

In [9]:
base_raster_name = os.path.join(out_folder, "TT_POP_%s.tif")
base_urban_raster_name = os.path.join(out_folder, "TT_POP_%s_URBAN.tif")
out_meta = pop_raster.meta.copy()
out_rasters = [out_pop_surface, urban_pop_raster]
for thresh in thresholds:
    cur_out_file = base_raster_name % thresh
    cur_out_urban_file = base_urban_raster_name % thresh
    out_rasters.append(cur_out_file)
    out_rasters.append(cur_out_urban_file)
    if not os.path.exists(cur_out_file) or not os.path.exists(cur_out_urban_file):
        cur_tt_d = (tt_d < thresh).astype('int')
        cur_pop = pop_d * cur_tt_d
        cur_urban_pop = urban_pop_d * cur_tt_d    
        with rasterio.open(cur_out_file, 'w', **meta) as outR:
            outR.write(cur_pop)

        with rasterio.open(cur_out_urban_file, 'w', **meta) as outR:
            outR.write(cur_urban_pop)

In [10]:
all_res = {}
try:
    del(compiled)
except:
    pass

for pop_R_file in out_rasters:
    res = rMisc.zonalStats(inB1, pop_R_file, minVal=0)
    name = os.path.basename(pop_R_file).replace(".tif", "")
    cols = ["%s_%s" % (name, x) for x in ['SUM','MIN','MAX','MEAN']]
    res = pd.DataFrame(res, columns=cols)
    all_res[name] = res
    try:
        final = final.join(res)
    except:
        final = res
    

In [11]:
output = final.filter(regex="SUM")
output['NAME'] = inB1['WB_ADM1_NA']
output['CODE'] = inB1['WB_ADM1_CO']
output.to_csv(os.path.join(out_folder, "COMBINED_ZONAL_STATS.csv"))

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
output['NAME'] = inB1['WB_ADM1_NA']
output.head()

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,POP_2020_NEW_SUM,URBAN_POP_SUM,TT_POP_30_SUM,TT_POP_30_URBAN_SUM,TT_POP_60_SUM,TT_POP_60_URBAN_SUM,TT_POP_120_SUM,TT_POP_120_URBAN_SUM,TT_POP_180_SUM,TT_POP_180_URBAN_SUM,NAME,CODE
0,8.432289e+06,1.509836e+06,3.988960e+06,1.486072e+06,5.236531e+06,1.507226e+06,6.518365e+06,1.509011e+06,7.227903e+06,1.509011e+06,NaN,NaN
1,5.004354e+06,2.089504e+06,2.482854e+06,1.547257e+06,3.520969e+06,1.819284e+06,4.406375e+06,1.980124e+06,4.749512e+06,2.034872e+06,NaN,NaN
2,2.062503e+06,2.017639e+06,2.060096e+06,2.018816e+06,2.062464e+06,2.018816e+06,2.063186e+06,2.018816e+06,2.063186e+06,2.018816e+06,NaN,NaN
3,3.248697e+07,2.495310e+07,2.639326e+07,2.303101e+07,2.879844e+07,2.408556e+07,3.044407e+07,2.468825e+07,3.111876e+07,2.486437e+07,NaN,NaN
4,1.278862e+08,1.087255e+08,1.206916e+08,1.070120e+08,1.254969e+08,1.084583e+08,1.273570e+08,1.086927e+08,1.276281e+08,1.087036e+08,NaN,NaN
